In [ ]:
import pandas as pd
import openai
import re
import json
import datasets
import transformers, peft
import numpy as np
import FlagEmbedding
import random

from FlagEmbedding import BGEM3FlagModel
from tqdm import tqdm
from huggingface_hub import login
from datasets import load_dataset

# 1. QA 데이터셋 구축

In [ ]:
df_pack = pd.read_json("final_tuning_sample_1000.jsonl", lines=True)
df_pack

,drug_id,drug_name_key,drug_name,active_ingredient,pro_type,context_mode,qa_type_hint,source_chunk_ids,context,warn_total_chunks,context_len,context_len_bin,warn_bin,ingredient_key,ingredient_bucket
0,09a0b7c7451e,코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타(디클로페낙나트륨),[M050288]디클로페낙나트륨,일반의약품,warning,warning,[09a0b7c7451e-warn-1],코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,6,316,L0_400,W5_7,M050288,HEAD
1,62227c2b97f5,하이로손로션2.5%(히드로코르티손),하이로손로션2.5%(히드로코르티손),[M040818]히드로코르티손,일반의약품,warning,warning,[62227c2b97f5-warn-1],하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,11,343,L0_400,W8p,M040818,HEAD
2,e227ace54e1f,케이포비돈아오다인액10%(포비돈요오드),케이포비돈아오다인액10%(포비돈요오드),[M244708]포비돈요오드,일반의약품,warning,warning,[e227ace54e1f-warn-5],케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...,11,268,L0_400,W8p,M244708,HEAD
3,01a538aa8035,유카인크림(리도카인),유카인크림(리도카인),[M040131]리도카인,일반의약품,warning,warning,[01a538aa8035-warn-5],유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,5,135,L0_400,W5_7,M040131,HEAD
4,7304a9a6f07c,케로팝경고제(케토프로펜),케로팝경고제(케토프로펜),[M051464]케토프로펜,일반의약품,warning,warning,[7304a9a6f07c-warn-1],케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,5,406,L401_800,W5_7,M051464,HEAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,c754c587d940,카프맥스정,카프맥스정,[M040353]아세트아미노펜|[M223155]에페드린염산염|[M223183]카페인무수물,일반의약품,warning,warning,[c754c587d940-warn-4],카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,8,749,L401_800,W8p,M040353,HEAD
996,849ec4fc5a8a,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),[M278007]멜라토닌,일반의약품,warning,warning,[849ec4fc5a8a-warn-4],슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n4...,6,193,L0_400,W5_7,M278007,TAIL
997,11a4afc7bec0,알파니푸록사지드캡슐(수출용),알파니푸록사지드캡슐(수출용),[M200244]니푸록사지드,일반의약품,warning,warning,[11a4afc7bec0-warn-2],알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르기...,4,55,L0_400,W3_4,M200244,TAIL
998,2b54290a89ea,징카민정80mg(은행엽건조엑스),징카민정80mg(은행엽건조엑스),[M088948]은행엽건조엑스,일반의약품,warning,warning,[2b54290a89ea-warn-5],징카민정80mg(은행엽건조엑스) :\n\n### 주의사항(1)\n5. 저장상의 주의...,5,115,L0_400,W5_7,M088948,HEAD


In [ ]:
client = openai.OpenAI(api_key="apikey")

In [ ]:
user_prompt_for_question = df_pack["context"].tolist()

In [ ]:
system_prompt_for_question = """너는 일반의약품 첨부문서를 기반으로 QA 데이터셋의 '질문'을 만든다. 
    규칙: 
    1) 반드시 CONTEXT 안의 정보로만 질문을 만든다(추측/지식 추가 금지). 
    2) 질문은 1개만 생성한다. 
    3) 질문은 답을 포함하거나 암시하지 않는다. 
    4) 출력은 반드시 JSON 한 개만 반환한다(다른 텍스트 금지). 
    6) 20자 이내로 답할 수 있는 내용으로 묻는다. 
    7) 질문은 반드시 CONTEXT에 명시된 약물명으로 시작한다
    8) 질문에는 '이 약', '해당 약', '주의사항은?' 같은 모호한 표현을 쓰지 않는다.
    9) 질문은 반드시 CONTEXT의 특정 항목/문장을 직접 지칭하도록 구체적으로 만든다.
     좋은예) 
     - 코디톱쿨플라스타는 임산부가 복용해도 되나요?
     - 타이레놀은 음주 전후에 복용해도 되나요?
     - 까스퀵은 간이 약한 사람이 복용해도 괜찮나요?

     금지예시)
     - 탁센이부프로펜의 주의사항은?
     - 리메부틴 복용시 주의해야 할 점은?
     - 콜드퀵의 효과는?
    """

- 5개로 테스트

In [ ]:
result_lst = [] 

for user_prompt in tqdm(user_prompt_for_question[:5]): 
    response = client.chat.completions.create( 
        model="gpt-4o-mini", 
        messages=[ {"role": "system", "content": system_prompt_for_question}, 
                  {"role": "user", "content": user_prompt + '질문:'} ], temperature=0 
    ) 
    result_lst.append(response.choices[0].message.content) 

100%|██████████| 5/5 [00:03<00:00,  1.43it/s]


In [ ]:
result_lst

['{\n  "question": "코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?"\n}',
 '{\n  "question": "하이로손로션2.5%는 고막 천공 환자가 사용할 수 있나요?"\n}',
 '{\n  "question": "케이포비돈아오다인액10%는 리튬치료와 병용해도 되나요?"\n}',
 '{\n  "question": "유카인크림은 어떻게 보관해야 하나요?"\n}',
 '{\n  "question": "케로팝경고제는 아스피린 천식 환자가 사용해도 되나요?"\n}']

In [ ]:
result_lst = [] 

for user_prompt in tqdm(user_prompt_for_question): 
    response = client.chat.completions.create( 
        model="gpt-4o-mini", 
        messages=[ {"role": "system", "content": system_prompt_for_question}, 
                  {"role": "user", "content": user_prompt + '질문:'} ], temperature=0 
    ) 
    result_lst.append(response.choices[0].message.content) 

100%|██████████| 1000/1000 [12:09<00:00,  1.37it/s]


In [ ]:
result_lst

['{\n  "question": "코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?"\n}',
 '{\n  "question": "하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?"\n}',
 '{\n  "question": "케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?"\n}',
 '{\n  "question": "유카인크림은 어떻게 보관해야 하나요?"\n}',
 '{\n  "question": "케로팝경고제는 15세 미만 소아에게 사용 가능한가?"\n}',
 '{\n  "question": "진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?"\n}',
 '{\n  "question": "파비스암브록솔염산염정은 어떻게 보관해야 하나요?"\n}',
 '{\n  "question": "알피디연질캡슐은 수유 중 복용해도 되나요?"\n}',
 '{\n  "question": "건아이점안액 사용 후 증상이 72시간 이상 지속되면 어떻게 해야 하나요?"\n}',
 '```json\n{\n  "question": "태극케토코나졸크림은 임산부가 사용해도 되나요?"\n}\n```',
 '{\n  "question": "다리아정은 증상 개선이 없을 때 어떻게 해야 하나요?"\n}',
 '{\n  "question": "씨이릴리프점안액은 하루 몇 회 점안하나요?"\n}',
 '{\n  "question": "치큐어캡슐 복용 후 소화장애가 나타나면 어떻게 해야 하나요?"\n}',
 '```json\n{\n  "question": "마이톡톡케어점안액은 소프트콘택트렌즈 착용 시 사용 가능한가요?"\n}\n```',
 '{\n  "question": "헤모데이캡슐은 어떻게 보관해야 하나요?"\n}',
 '{\n  "question": "광동금탕액은 고혈압 환자가 복용해도 되나요?"\n}',
 '{\n  "question": "치퀵정은 주성분에 과민반응이 있는 사람은 복용해도 되나요?"\n}',
 '{\n  "question": "폴락스산4그

In [ ]:
def safe_get_question(s):
    if s is None:
        return None

    s = str(s).strip()
    if not s:
        return None

    try:
        obj = json.loads(s)
        if isinstance(obj, dict) and "question" in obj:
            return obj["question"]
    except Exception:
        pass

    m = re.search(r"\{.*\}", s, flags=re.S)
    if m:
        try:
            obj = json.loads(m.group(0))
            if isinstance(obj, dict) and "question" in obj:
                return obj["question"]
        except Exception:
            pass

    return None

questions = [safe_get_question(x) for x in result_lst]

In [ ]:
df_question = df_pack[["context", "qa_type_hint", "drug_id", "drug_name_key", "drug_name"]].copy()
df_question["question"] = questions
df_question = df_question.reset_index(drop=True)
df_question

,context,qa_type_hint,drug_id,drug_name_key,drug_name,question
0,코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,warning,09a0b7c7451e,코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?
1,하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,warning,62227c2b97f5,하이로손로션2.5%(히드로코르티손),하이로손로션2.5%(히드로코르티손),하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?
2,케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...,warning,e227ace54e1f,케이포비돈아오다인액10%(포비돈요오드),케이포비돈아오다인액10%(포비돈요오드),케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?
3,유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,warning,01a538aa8035,유카인크림(리도카인),유카인크림(리도카인),유카인크림은 어떻게 보관해야 하나요?
4,케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,warning,7304a9a6f07c,케로팝경고제(케토프로펜),케로팝경고제(케토프로펜),케로팝경고제는 15세 미만 소아에게 사용 가능한가?
...,...,...,...,...,...,...
995,카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,warning,c754c587d940,카프맥스정,카프맥스정,카프맥스정은 임부가 복용해도 되나요?
996,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n4...,warning,849ec4fc5a8a,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),슬리프웰구강붕해필름은 임부가 복용해도 되나요?
997,알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르기...,warning,11a4afc7bec0,알파니푸록사지드캡슐(수출용),알파니푸록사지드캡슐(수출용),알파니푸록사지드캡슐의 부작용은 무엇인가요?
998,징카민정80mg(은행엽건조엑스) :\n\n### 주의사항(1)\n5. 저장상의 주의...,warning,2b54290a89ea,징카민정80mg(은행엽건조엑스),징카민정80mg(은행엽건조엑스),징카민정80mg은 어떻게 보관해야 하나요?


In [ ]:
df_question[df_question["question"].isna()]

,context,qa_type_hint,drug_id,drug_name_key,drug_name,question


In [ ]:
df_question = df_question[["drug_id", "drug_name_key", "drug_name", "qa_type_hint", "context", "question"]]

In [ ]:
df_question

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question
0,09a0b7c7451e,코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타(디클로페낙나트륨),warning,코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?
1,62227c2b97f5,하이로손로션2.5%(히드로코르티손),하이로손로션2.5%(히드로코르티손),warning,하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?
2,e227ace54e1f,케이포비돈아오다인액10%(포비돈요오드),케이포비돈아오다인액10%(포비돈요오드),warning,케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?
3,01a538aa8035,유카인크림(리도카인),유카인크림(리도카인),warning,유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,유카인크림은 어떻게 보관해야 하나요?
4,7304a9a6f07c,케로팝경고제(케토프로펜),케로팝경고제(케토프로펜),warning,케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,케로팝경고제는 15세 미만 소아에게 사용 가능한가?
...,...,...,...,...,...,...
995,c754c587d940,카프맥스정,카프맥스정,warning,카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,카프맥스정은 임부가 복용해도 되나요?
996,849ec4fc5a8a,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),warning,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n4...,슬리프웰구강붕해필름은 임부가 복용해도 되나요?
997,11a4afc7bec0,알파니푸록사지드캡슐(수출용),알파니푸록사지드캡슐(수출용),warning,알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르기...,알파니푸록사지드캡슐의 부작용은 무엇인가요?
998,2b54290a89ea,징카민정80mg(은행엽건조엑스),징카민정80mg(은행엽건조엑스),warning,징카민정80mg(은행엽건조엑스) :\n\n### 주의사항(1)\n5. 저장상의 주의...,징카민정80mg은 어떻게 보관해야 하나요?


- hugging face 업로드

In [ ]:
dataset = datasets.Dataset.from_pandas(df_question)

In [ ]:
dataset.push_to_hub("otc-mrc-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/eddyfox8812/otc-mrc-question/commit/ee9bb887ed24aa370ed4ef92cb9b99470a7b30df', commit_message='Upload dataset', commit_description='', oid='ee9bb887ed24aa370ed4ef92cb9b99470a7b30df', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eddyfox8812/otc-mrc-question', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eddyfox8812/otc-mrc-question'), pr_revision=None, pr_num=None)

# 2. negative sample 생성

In [ ]:
dataset = load_dataset("eddyfox8812/otc-mrc-question")

df = dataset['train'].to_pandas()
print(df.head())

        drug_id          drug_name_key              drug_name qa_type_hint  \
0  09a0b7c7451e     코디톱쿨플라스타(디클로페낙나트륨)     코디톱쿨플라스타(디클로페낙나트륨)      warning   
1  62227c2b97f5    하이로손로션2.5%(히드로코르티손)    하이로손로션2.5%(히드로코르티손)      warning   
2  e227ace54e1f  케이포비돈아오다인액10%(포비돈요오드)  케이포비돈아오다인액10%(포비돈요오드)      warning   
3  01a538aa8035            유카인크림(리도카인)            유카인크림(리도카인)      warning   
4  7304a9a6f07c          케로팝경고제(케토프로펜)          케로팝경고제(케토프로펜)      warning   

                                             context  \
0  코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...   
1  하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...   
2  케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...   
3  유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...   
4  케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...   

                           question  
0   코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?  
1  하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?  
2   케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?  
3              유카인크림은 어떻게 보관해야 하나요?  
4   

In [ ]:
df[['context', 'question']]

,context,question
0,코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?
1,하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?
2,케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?
3,유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,유카인크림은 어떻게 보관해야 하나요?
4,케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,케로팝경고제는 15세 미만 소아에게 사용 가능한가?
...,...,...
995,카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,카프맥스정은 임부가 복용해도 되나요?
996,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n4...,슬리프웰구강붕해필름은 임부가 복용해도 되나요?
997,알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르기...,알파니푸록사지드캡슐의 부작용은 무엇인가요?
998,징카민정80mg(은행엽건조엑스) :\n\n### 주의사항(1)\n5. 저장상의 주의...,징카민정80mg은 어떻게 보관해야 하나요?


- 임베딩

In [ ]:
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

...

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

In [ ]:
def get_embeddings(sentences, batch_size = 32) :
  return model.encode(sentences, batch_size=batch_size, max_length=512)['dense_vecs']

In [ ]:
print("Generating context embeddings...")
context_embeddings = get_embeddings(df['context'].to_list())

Generating context embeddings...


pre tokenize: 100%|██████████| 32/32 [00:00<00:00, 305.42it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 32/32 [00:01<00:00, 24.52it/s]


- 유사도 분석 후 top-4 선택

In [ ]:
negative_samples_list = []

print("Finding negative samples...")
for i in tqdm(range(len(df)), total=len(df), dynamic_ncols=True) :
  question = df.iloc[i]['question']

  question_embedding = get_embeddings([question])[0]

  similarities = np.dot(context_embeddings, question_embedding)

  top_n_indices = similarities.argsort()[::-1]
  top_n_indices = [idx for idx in top_n_indices if idx != i][:4]

  negative_samples = [df['context'].iloc[idx] for idx in top_n_indices]
  negative_samples_list.append(negative_samples)

Finding negative samples...


100%|██████████| 1000/1000 [00:38<00:00, 25.69it/s]


In [ ]:
df['negative_samples'] = negative_samples_list

print(df.head())

        drug_id          drug_name_key              drug_name qa_type_hint  \
0  09a0b7c7451e     코디톱쿨플라스타(디클로페낙나트륨)     코디톱쿨플라스타(디클로페낙나트륨)      warning   
1  62227c2b97f5    하이로손로션2.5%(히드로코르티손)    하이로손로션2.5%(히드로코르티손)      warning   
2  e227ace54e1f  케이포비돈아오다인액10%(포비돈요오드)  케이포비돈아오다인액10%(포비돈요오드)      warning   
3  01a538aa8035            유카인크림(리도카인)            유카인크림(리도카인)      warning   
4  7304a9a6f07c          케로팝경고제(케토프로펜)          케로팝경고제(케토프로펜)      warning   

                                             context  \
0  코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...   
1  하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...   
2  케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...   
3  유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...   
4  케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...   

                           question  \
0   코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?   
1  하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?   
2   케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?   
3              유카인크림은 어떻게 보관해야 하나요?   

In [ ]:
idx = 5

print("질문 :", df['question'].loc[idx])
print("정답 문서 : ", df['context'].loc[idx])
print("네거티브 문서: ", df['negative_samples'].loc[idx])
print("네거티브 문서의 갯수: ", len(df['negative_samples'].loc[idx]))

질문 : 진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?
정답 문서 :  진코신정80밀리그램(은행엽건조엑스) :

### 주의사항(1)
2. 다음과 같은 사람은 이 약을 복용(사용)하기 전에 의사, 치과의사, 약사와 상의할 것
1) 출혈 장애가 있거나 출혈의 위험을 증가시키는 약물(항응고제(예: 펜프로쿠몬, 와파린)), 항혈소판제(예:클
로피도그렐, 아세틸살리실산 및 기타 비스테로이드성 항염증제))을 복용하고 있는 환자
2) 수유부(모유로 이행여부가 알려지지 않음)
네거티브 문서:  ['징코하이정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n3. 다음과 같은 사람은 이 약을 복용(사용)하기 전에 의사, 치과의사, 약사와 상의할 것\n1) 출혈 장애가 있거나 출혈의 위험을 증가시키는 약물을 복용(사용)하고 있는 환자는 의학적 조언을 구해야 \n한다.\n2) 수유부(모유로 이행여부가 알려지지 않음)\n3) 이 약을 복용(사용)하여 뇌전증 환자의 추가 발작이 촉진되는 것을 배제할 수 없다.', '징코플로우정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n3. 다음과 같은 사람은 이 약을 복용(사용)하기 전에 의사, 치과의사, 약사와 상의할 것\n1) 출혈 장애가 있거나 출혈의 위험을 증가시키는 약물을 복용(사용)하고 있는 환자는 의학적 조언을 구해야 \n한다.\n2) 수유부(모유로 이행여부가 알려지지 않음)\n3) 이 약을 복용(사용)하여 뇌전증 환자의 추가 발작이 촉진되는 것을 배제할 수 없다.', '기넥신메모케어캡슐 :\n\n### 주의사항(1)\n2. 다음과 같은 사람은 이 약을 복용(사용)하기 전에 의사, 치과의사, 약사와 상의할 것\n1) 출혈 장애가 있거나 출혈의 위험을 증가시키는 약물(항응고제(예: 펜프로쿠몬, 와파린)), 항혈소판제(예:클\n로피도그렐, 아세틸살리실산 및 기타 비스테로이드성 항염증제))을 복용하고 있는 환자\n2) 수유부(모유로 이행여부가 알려지지 않음)\n3) 이 약은 황색5호(선셋옐로우

In [ ]:
df

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples
0,09a0b7c7451e,코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타(디클로페낙나트륨),warning,코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...
1,62227c2b97f5,하이로손로션2.5%(히드로코르티손),하이로손로션2.5%(히드로코르티손),warning,하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...
2,e227ace54e1f,케이포비돈아오다인액10%(포비돈요오드),케이포비돈아오다인액10%(포비돈요오드),warning,케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?,[코오푸론과립(마행감석탕) :\n\n### 주의사항(1)\n4. 기타 이 약의 복용...
3,01a538aa8035,유카인크림(리도카인),유카인크림(리도카인),warning,유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,유카인크림은 어떻게 보관해야 하나요?,[엑스케인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주...
4,7304a9a6f07c,케로팝경고제(케토프로펜),케로팝경고제(케토프로펜),warning,케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,케로팝경고제는 15세 미만 소아에게 사용 가능한가?,[크린팝카타플라스마 :\n\n### 주의사항(1)\n1. 다음 환자(부위)에는 투여...
...,...,...,...,...,...,...,...
995,c754c587d940,카프맥스정,카프맥스정,warning,카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,카프맥스정은 임부가 복용해도 되나요?,"[레피즈캡슐(니푸록사지드) :\n\n### 주의사항(1)\n6. 임부, 수유부, 가..."
996,849ec4fc5a8a,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),warning,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n4...,슬리프웰구강붕해필름은 임부가 복용해도 되나요?,[시어스피록시캄겔 :\n\n### 주의사항(1)\n5. 임부 및 수유부에 대한 투여...
997,11a4afc7bec0,알파니푸록사지드캡슐(수출용),알파니푸록사지드캡슐(수출용),warning,알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르기...,알파니푸록사지드캡슐의 부작용은 무엇인가요?,[알파우르소데옥시콜산연질캡슐100밀리그램(수출용) :\n\n### 주의사항(1)\n...
998,2b54290a89ea,징카민정80mg(은행엽건조엑스),징카민정80mg(은행엽건조엑스),warning,징카민정80mg(은행엽건조엑스) :\n\n### 주의사항(1)\n5. 저장상의 주의...,징카민정80mg은 어떻게 보관해야 하나요?,[징큐라민정(은행엽건조엑스) :\n\n### 주의사항(1)\n5. 저장상의 주의사항...


- 업로드

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset.push_to_hub("otc-mrc-bge-m3-negatve-samples")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/eddyfox8812/otc-mrc-bge-m3-negatve-samples/commit/00a154c9cafa1aaf605dd55410c40552bdb7cbb1', commit_message='Upload dataset', commit_description='', oid='00a154c9cafa1aaf605dd55410c40552bdb7cbb1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eddyfox8812/otc-mrc-bge-m3-negatve-samples', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eddyfox8812/otc-mrc-bge-m3-negatve-samples'), pr_revision=None, pr_num=None)

# 3. 답변데이터 생성

## 3-1. 일반답변/no answer 분할

In [ ]:
dataset = load_dataset("eddyfox8812/otc-mrc-bge-m3-negatve-samples")

df = dataset['train'].to_pandas()

print(df.head())

        drug_id          drug_name_key              drug_name qa_type_hint  \
0  09a0b7c7451e     코디톱쿨플라스타(디클로페낙나트륨)     코디톱쿨플라스타(디클로페낙나트륨)      warning   
1  62227c2b97f5    하이로손로션2.5%(히드로코르티손)    하이로손로션2.5%(히드로코르티손)      warning   
2  e227ace54e1f  케이포비돈아오다인액10%(포비돈요오드)  케이포비돈아오다인액10%(포비돈요오드)      warning   
3  01a538aa8035            유카인크림(리도카인)            유카인크림(리도카인)      warning   
4  7304a9a6f07c          케로팝경고제(케토프로펜)          케로팝경고제(케토프로펜)      warning   

                                             context  \
0  코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...   
1  하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...   
2  케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...   
3  유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...   
4  케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...   

                           question  \
0   코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?   
1  하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?   
2   케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?   
3              유카인크림은 어떻게 보관해야 하나요?   

In [ ]:
def sample_general_noanswer(
    df_q: pd.DataFrame,
    general_n: int = 667,
    noanswer_n: int = 333,
    seed: int = 42,
    mode_col: str = "qa_type_hint",
) -> pd.DataFrame:

    assert general_n + noanswer_n == len(df_q)
    df = df_q.copy().reset_index(drop=True)
    rng = np.random.default_rng(seed)

    total = len(df)
    n_dosage = int((df[mode_col] == "dosage").sum())
    n_warning = int((df[mode_col] == "warning").sum())

    general_dosage = int(round(general_n * (n_dosage / total)))
    general_warning = general_n - general_dosage

    idx_d = df.index[df[mode_col] == "dosage"].to_numpy()
    idx_w = df.index[df[mode_col] == "warning"].to_numpy()

    general_idx = np.concatenate([
        rng.choice(idx_d, size=general_dosage, replace=False),
        rng.choice(idx_w, size=general_warning, replace=False)
    ])
    rng.shuffle(general_idx)

    df["label_type"] = "noanswer"
    df.loc[general_idx, "label_type"] = "general"

    return df


df_train = sample_general_noanswer(df, seed=42)
print(df_train["label_type"].value_counts())
print(df_train.groupby(["label_type","qa_type_hint"]).size())

label_type
general     667
noanswer    333
Name: count, dtype: int64
label_type  qa_type_hint
general     dosage           67
            warning         600
noanswer    dosage           33
            warning         300
dtype: int64


In [ ]:
df_general = df_train[df_train['label_type'] == 'general']
df_general

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type
2,e227ace54e1f,케이포비돈아오다인액10%(포비돈요오드),케이포비돈아오다인액10%(포비돈요오드),warning,케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?,[코오푸론과립(마행감석탕) :\n\n### 주의사항(1)\n4. 기타 이 약의 복용...,general
4,7304a9a6f07c,케로팝경고제(케토프로펜),케로팝경고제(케토프로펜),warning,케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,케로팝경고제는 15세 미만 소아에게 사용 가능한가?,[크린팝카타플라스마 :\n\n### 주의사항(1)\n1. 다음 환자(부위)에는 투여...,general
5,f4c544a98592,진코신정80밀리그램(은행엽건조엑스),진코신정80밀리그램(은행엽건조엑스),warning,진코신정80밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n2. 다음과 같...,진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?,[징코하이정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n3. 다음...,general
6,1eae41b300f9,파비스암브록솔염산염정(수출용),파비스암브록솔염산염정(수출용),warning,파비스암브록솔염산염정(수출용) :\n\n### 주의사항(1)\n6. 저장상의 주의사...,파비스암브록솔염산염정은 어떻게 보관해야 하나요?,[파믹스징코정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n6. 저...,general
7,3b49b5e28d21,알피디연질캡슐1마이크로그램(알파칼시돌),알피디연질캡슐1마이크로그램(알파칼시돌),warning,알피디연질캡슐1마이크로그램(알파칼시돌) :\n\n### 주의사항(1)\n3. 이 약...,알피디연질캡슐은 수유 중 복용해도 되나요?,[이지페인릴리프연질캡슐 :\n\n### 주의사항(1)\n8. 임부 및 수유부에 대한...,general
...,...,...,...,...,...,...,...,...
993,a50c93432640,마이노신캡슐(배농산급탕),마이노신캡슐(배농산급탕),warning,마이노신캡슐(배농산급탕) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,마이노신캡슐은 갈락토오스 불내성 환자가 복용해도 되나요?,[브로신큐정 :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 이 약을 복용...,general
994,83916b730ddb,벡크정,벡크정,dosage,벡크정 :\n\n### 용법용량(1)\n용법용량\n만 8세 이상 및 성인 : 1회 ...,벡크정은 만 8세 이상에게 어떻게 복용하나요?,[바이타비정 :\n\n### 용법용량(1)\n용법용량\n만 8세 이상 어린이 및 성...,general
996,849ec4fc5a8a,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),warning,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n4...,슬리프웰구강붕해필름은 임부가 복용해도 되나요?,[시어스피록시캄겔 :\n\n### 주의사항(1)\n5. 임부 및 수유부에 대한 투여...,general
997,11a4afc7bec0,알파니푸록사지드캡슐(수출용),알파니푸록사지드캡슐(수출용),warning,알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르기...,알파니푸록사지드캡슐의 부작용은 무엇인가요?,[알파우르소데옥시콜산연질캡슐100밀리그램(수출용) :\n\n### 주의사항(1)\n...,general


In [ ]:
df_no = df_train[df_train["label_type"] == "noanswer"]
df_no

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type
0,09a0b7c7451e,코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타(디클로페낙나트륨),warning,코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...,noanswer
1,62227c2b97f5,하이로손로션2.5%(히드로코르티손),하이로손로션2.5%(히드로코르티손),warning,하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...,noanswer
3,01a538aa8035,유카인크림(리도카인),유카인크림(리도카인),warning,유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,유카인크림은 어떻게 보관해야 하나요?,[엑스케인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주...,noanswer
11,e79531ef699d,씨이릴리프점안액(수출용),씨이릴리프점안액(수출용),dosage,"씨이릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1일 1-4회,...",씨이릴리프점안액은 하루 몇 회 점안하나요?,[어드밴스드릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1회 1...,noanswer
13,9aabec0ade2d,마이톡톡케어점안액,마이톡톡케어점안액,warning,마이톡톡케어점안액 :\n\n### 주의사항(1)\n사용상의주의사항\n1) 다음과 같...,마이톡톡케어점안액은 소프트콘택트렌즈 착용 시 사용 가능한가요?,[아이미루40이엑스점안액 :\n\n### 주의사항(1)\n3. 이 약을 사용하는 동...,noanswer
...,...,...,...,...,...,...,...,...
983,29502210f1b0,경남나프록센연질캡슐,경남나프록센연질캡슐,dosage,경남나프록센연질캡슐 :\n\n### 용법용량(1)\n용법용량\n1. 류마티양 관절염...,경남나프록센연질캡슐의 급성통풍 용법은?,[나프원큐연질캡슐(나프록센) :\n\n### 주의사항(1)\n2. 다음 환자에는 투...,noanswer
989,79bccd3248e9,폴리아린락토정,폴리아린락토정,warning,폴리아린락토정 :\n\n### 주의사항(1)\n3. 다음과 같은 사람(경우)은 이 ...,폴리아린락토정은 알레르기 병력이 있는 사람에게 어떻게 해야 하나요?,[락토란정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람(경우) 이 약의 ...,noanswer
992,c33a93c16f09,휘니볼(반하사심탕엑스과립),휘니볼(반하사심탕엑스과립),dosage,휘니볼(반하사심탕엑스과립) :\n\n### 용법용량(1)\n용법용량\n보통 성인 1...,휘니볼의 성인 복용량은 얼마인가요?,[폰트비정 :\n\n### 용법용량(1)\n용법용량\n만 12세 이상 및 성인 : ...,noanswer
995,c754c587d940,카프맥스정,카프맥스정,warning,카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,카프맥스정은 임부가 복용해도 되나요?,"[레피즈캡슐(니푸록사지드) :\n\n### 주의사항(1)\n6. 임부, 수유부, 가...",noanswer


## 3-2. 일반 답변데이터 생성

In [ ]:
df_general = df_general.reset_index(drop=True)
df_general

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type
0,e227ace54e1f,케이포비돈아오다인액10%(포비돈요오드),케이포비돈아오다인액10%(포비돈요오드),warning,케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?,[코오푸론과립(마행감석탕) :\n\n### 주의사항(1)\n4. 기타 이 약의 복용...,general
1,7304a9a6f07c,케로팝경고제(케토프로펜),케로팝경고제(케토프로펜),warning,케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,케로팝경고제는 15세 미만 소아에게 사용 가능한가?,[크린팝카타플라스마 :\n\n### 주의사항(1)\n1. 다음 환자(부위)에는 투여...,general
2,f4c544a98592,진코신정80밀리그램(은행엽건조엑스),진코신정80밀리그램(은행엽건조엑스),warning,진코신정80밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n2. 다음과 같...,진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?,[징코하이정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n3. 다음...,general
3,1eae41b300f9,파비스암브록솔염산염정(수출용),파비스암브록솔염산염정(수출용),warning,파비스암브록솔염산염정(수출용) :\n\n### 주의사항(1)\n6. 저장상의 주의사...,파비스암브록솔염산염정은 어떻게 보관해야 하나요?,[파믹스징코정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n6. 저...,general
4,3b49b5e28d21,알피디연질캡슐1마이크로그램(알파칼시돌),알피디연질캡슐1마이크로그램(알파칼시돌),warning,알피디연질캡슐1마이크로그램(알파칼시돌) :\n\n### 주의사항(1)\n3. 이 약...,알피디연질캡슐은 수유 중 복용해도 되나요?,[이지페인릴리프연질캡슐 :\n\n### 주의사항(1)\n8. 임부 및 수유부에 대한...,general
...,...,...,...,...,...,...,...,...
662,a50c93432640,마이노신캡슐(배농산급탕),마이노신캡슐(배농산급탕),warning,마이노신캡슐(배농산급탕) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...,마이노신캡슐은 갈락토오스 불내성 환자가 복용해도 되나요?,[브로신큐정 :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 이 약을 복용...,general
663,83916b730ddb,벡크정,벡크정,dosage,벡크정 :\n\n### 용법용량(1)\n용법용량\n만 8세 이상 및 성인 : 1회 ...,벡크정은 만 8세 이상에게 어떻게 복용하나요?,[바이타비정 :\n\n### 용법용량(1)\n용법용량\n만 8세 이상 어린이 및 성...,general
664,849ec4fc5a8a,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용),warning,슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n4...,슬리프웰구강붕해필름은 임부가 복용해도 되나요?,[시어스피록시캄겔 :\n\n### 주의사항(1)\n5. 임부 및 수유부에 대한 투여...,general
665,11a4afc7bec0,알파니푸록사지드캡슐(수출용),알파니푸록사지드캡슐(수출용),warning,알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르기...,알파니푸록사지드캡슐의 부작용은 무엇인가요?,[알파우르소데옥시콜산연질캡슐100밀리그램(수출용) :\n\n### 주의사항(1)\n...,general


In [ ]:
samples = []
for context, negative_samples in zip(df_general['context'].to_list(), df_general['negative_samples'].to_list()):
  documents = [context] + list(negative_samples)
  random.shuffle(documents)
  samples.append(documents)

In [ ]:
user_prompts = []

for sample in samples:
  user_prompt = ''

  for idx, s in enumerate(sample):
    user_prompt = user_prompt + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [ ]:
print(user_prompts[0])

문서1:케이포비돈아오다인액10%(포비돈요오드) :

### 주의사항(1)
5. 상호작용
1) 리튬치료를 받고있는 환자에게는 이 약의 규칙적인 사용을 피한다. 리튬치료와 병용 할 경우 갑상선기능 
항진효과가 나타날 수 있다.

2) 클로르헥시딘, 설파디아진 ․은화합물, 알칼리, 수은을 함유한 제제와 병용할 경우 이 약의 살균효과가 감소
될 수 있으며 이는 변색을 통해 알 수 있다.
3) 효소성분의 상처치료제와 병용시 효소의 효과가 상실된다.
4) 이 약은 과산화수소와 병용하지 않는다.
---
문서2:코오푸론과립(마행감석탕) :

### 주의사항(1)
4. 기타 이 약의 복용시 주의할 사항
1) 칼륨함유제제, 감초함유제제, 글리시리진산 또는 그 염류 함유제제, 루프계 이뇨제(푸로세미드, 에타크린
산) 또는 티아지드계 이뇨제(트리클로르메티아지드)와 병용시 위알도스테론증이나 저칼륨혈증으로 인하여 
근병증이 나타나기 쉬우므로 신중히 복용할 것.
2) 정해진 용법·용량을 지킬 것.
3) 장기간 계속하여 복용하는 경우에는 의사, 한의사, 치과의사, 약사, 한약사와 상의할 것.
4) 마황 또는 에페드린 함유제제, MAO 억제제, 갑상선제제(티록신, 리오치오닌), 카테콜아민제제(에피네프
린, 이소프레날린), 크산틴계제제(테오필린, 디프로필린)와 병용 시 불면, 땀 과다, 빠른맥, 소변을 자주 봄, 
두근거림, 전신무력감, 정신흥분 등의 증상이 나타나기 쉬우므로 감량하는 등 신중히 복용할 것.
5) 어린이에게 복용시킬 경우에는 보호자의 지도·감독하에 복용시킬 것(과립제에 한함.)
---
문서3:노즈진정120밀리그램(펙소페나딘염산염) :

### 주의사항(1)
5. 상호작용
1) 이 약의 주성분인 펙소페나딘은 테르페나딘의 생체내 대사물로서 간장에서 대사되지 않는다. 따라서 간대
사를 통한 다른 약물과의 상호작용은 없다.
2) 케토코나졸이나 에리트로마이신과 병용투여하면 이 약의 혈중농도가 2∼3배는 상승되나, 이 변화는 신전
도상의 QT간격에 영향을 미치지 않는다. 또한, 이 약을

In [ ]:
user_prompt_for_answer = []
for multi_document_question, user_prompt in zip(df_general['question'].to_list(), user_prompts):
  user_prompt_for_answer.append('질문:' + multi_document_question + '\n' + user_prompt + '답변:')

In [ ]:
print(user_prompt_for_answer[0])

질문:케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?
문서1:케이포비돈아오다인액10%(포비돈요오드) :

### 주의사항(1)
5. 상호작용
1) 리튬치료를 받고있는 환자에게는 이 약의 규칙적인 사용을 피한다. 리튬치료와 병용 할 경우 갑상선기능 
항진효과가 나타날 수 있다.

2) 클로르헥시딘, 설파디아진 ․은화합물, 알칼리, 수은을 함유한 제제와 병용할 경우 이 약의 살균효과가 감소
될 수 있으며 이는 변색을 통해 알 수 있다.
3) 효소성분의 상처치료제와 병용시 효소의 효과가 상실된다.
4) 이 약은 과산화수소와 병용하지 않는다.
---
문서2:코오푸론과립(마행감석탕) :

### 주의사항(1)
4. 기타 이 약의 복용시 주의할 사항
1) 칼륨함유제제, 감초함유제제, 글리시리진산 또는 그 염류 함유제제, 루프계 이뇨제(푸로세미드, 에타크린
산) 또는 티아지드계 이뇨제(트리클로르메티아지드)와 병용시 위알도스테론증이나 저칼륨혈증으로 인하여 
근병증이 나타나기 쉬우므로 신중히 복용할 것.
2) 정해진 용법·용량을 지킬 것.
3) 장기간 계속하여 복용하는 경우에는 의사, 한의사, 치과의사, 약사, 한약사와 상의할 것.
4) 마황 또는 에페드린 함유제제, MAO 억제제, 갑상선제제(티록신, 리오치오닌), 카테콜아민제제(에피네프
린, 이소프레날린), 크산틴계제제(테오필린, 디프로필린)와 병용 시 불면, 땀 과다, 빠른맥, 소변을 자주 봄, 
두근거림, 전신무력감, 정신흥분 등의 증상이 나타나기 쉬우므로 감량하는 등 신중히 복용할 것.
5) 어린이에게 복용시킬 경우에는 보호자의 지도·감독하에 복용시킬 것(과립제에 한함.)
---
문서3:노즈진정120밀리그램(펙소페나딘염산염) :

### 주의사항(1)
5. 상호작용
1) 이 약의 주성분인 펙소페나딘은 테르페나딘의 생체내 대사물로서 간장에서 대사되지 않는다. 따라서 간대
사를 통한 다른 약물과의 상호작용은 없다.
2) 케토코나졸이나 에리트로마이신과 병용투여하면 이 약의 혈중농도가 2∼3배는 상승되나, 이 변화는

In [ ]:
system_prompt = """당신은 검색된 문서로부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 답변에서 참고자료의 내용을 인용한 경우, 답변 맨 끝에 인용한 문서 id를 추가하세요. 답변에 내용을 인용한 경우에만 작성해야 합니다. 인용한 게 없다면 쓰지마세요.
4. 문서 id 값은 검색 결과에서 가져온 것이며 두 개의 대괄호로 묶어야 하며 ref라는 키워드와 함께 작성해야 합니다. 예를 들어 특정 문단이 3번 문서에서 인용했다면 다음과 같이 작성하십시오.
예시) ~~~해서 했습니다. [[ref3]]
5. 주요 문장 또는 문단 뒤에 ref문서번호를 덧붙이는 것은 매우 중요합니다. 이는 반드시 지켜져야 합니다.
6. 문단 중간 중간에 인용이 들어가더라도 답변은 하나의 이어지는 평문으로 작성하십시오.
7. 주어진 문서는 검색 결과입니다. 따라서 질문과 연관없는 문서가 숨어있을 수 있으니 이를 인용하지 않도록 주의하십시오.
8. 검색 결과에 없는 내용은 답변으로 작성하지 마십시오. 가장 중요한 지시사항입니다. 따라서 인용 없이는 그 어떠한 설명도 제공할 수 없습니다.
9. 사용자는 한가지의 약물에 대해서만 질문합니다. 문서 내 약물을 정확히 찾아 답변하세요.
10. 반드시 사용자가 질문한 약물에 관한 문서만을 찾아 답변하세요. 이는 반드시 지켜져야 합니다.
11. 검색 결과는 모두 독립적인 문서들이며 반드시 연관되어져 있거나 순서대로 이어지는 문서들이 아닙니다.
"""

- 5개로 테스트

In [ ]:
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer[:5]):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:14<00:00,  2.83s/it]


In [ ]:
print(user_prompt_for_answer[0])

질문:케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?
문서1:케이포비돈아오다인액10%(포비돈요오드) :

### 주의사항(1)
5. 상호작용
1) 리튬치료를 받고있는 환자에게는 이 약의 규칙적인 사용을 피한다. 리튬치료와 병용 할 경우 갑상선기능 
항진효과가 나타날 수 있다.

2) 클로르헥시딘, 설파디아진 ․은화합물, 알칼리, 수은을 함유한 제제와 병용할 경우 이 약의 살균효과가 감소
될 수 있으며 이는 변색을 통해 알 수 있다.
3) 효소성분의 상처치료제와 병용시 효소의 효과가 상실된다.
4) 이 약은 과산화수소와 병용하지 않는다.
---
문서2:코오푸론과립(마행감석탕) :

### 주의사항(1)
4. 기타 이 약의 복용시 주의할 사항
1) 칼륨함유제제, 감초함유제제, 글리시리진산 또는 그 염류 함유제제, 루프계 이뇨제(푸로세미드, 에타크린
산) 또는 티아지드계 이뇨제(트리클로르메티아지드)와 병용시 위알도스테론증이나 저칼륨혈증으로 인하여 
근병증이 나타나기 쉬우므로 신중히 복용할 것.
2) 정해진 용법·용량을 지킬 것.
3) 장기간 계속하여 복용하는 경우에는 의사, 한의사, 치과의사, 약사, 한약사와 상의할 것.
4) 마황 또는 에페드린 함유제제, MAO 억제제, 갑상선제제(티록신, 리오치오닌), 카테콜아민제제(에피네프
린, 이소프레날린), 크산틴계제제(테오필린, 디프로필린)와 병용 시 불면, 땀 과다, 빠른맥, 소변을 자주 봄, 
두근거림, 전신무력감, 정신흥분 등의 증상이 나타나기 쉬우므로 감량하는 등 신중히 복용할 것.
5) 어린이에게 복용시킬 경우에는 보호자의 지도·감독하에 복용시킬 것(과립제에 한함.)
---
문서3:노즈진정120밀리그램(펙소페나딘염산염) :

### 주의사항(1)
5. 상호작용
1) 이 약의 주성분인 펙소페나딘은 테르페나딘의 생체내 대사물로서 간장에서 대사되지 않는다. 따라서 간대
사를 통한 다른 약물과의 상호작용은 없다.
2) 케토코나졸이나 에리트로마이신과 병용투여하면 이 약의 혈중농도가 2∼3배는 상승되나, 이 변화는

In [ ]:
result_lst[0]

'케이포비돈아오다인액10%는 리튬치료를 받고 있는 환자에게 규칙적인 사용을 피해야 하며, 리튬치료와 병용할 경우 갑상선기능 항진효과가 나타날 수 있습니다. 따라서 리튬치료와 병용하는 것은 권장되지 않습니다[[ref1]].'

In [ ]:
print(user_prompt_for_answer[1])

질문:케로팝경고제는 15세 미만 소아에게 사용 가능한가?
문서1:크린팝카타플라스마 :

### 주의사항(1)
1. 다음 환자(부위)에는 투여하지 말 것.
1) 30개월 이하의 유아
2) 습진, 옻 등에 의한 피부염, 상처부위
---
문서2:크레아베이직정 :

### 주의사항(1)
사용상의주의사항
1) 다음과 같은 사람은 이 약을 복용하지 말 것.
만 7세 이하의 어린이
2) 다음과 같은 사람은 이 약을 복용하기 전에 의사, 치과의사, 약사와 상의할 것.
(1) 본인 또는 가족이 알레르기 체질인 사람
(2) 임부 또는 임신하고 있을 가능성이 있는 여성
(3) 다른 약물을 복용하고 있는 사람
3) 다음과 같은 사람(경우) 이 약의 복용을 즉각 중지하고 의사,치과의사,약사와 상의할 것. 상담시 이 첨부문
서를 소지할 것.
2주 정도 투여하여도 증상의 개선이 없을 경우
4) 기타 이 약의 복용시 주의할 사항
정해진 용법 ․용량을 잘 지킬 것.
5) 저장상의 주의사항
(1) 어린이의 손에 닿지 않는 곳에 보관할 것.
(2) 직사광선을 피하고 될 수 있는 한 습기가 적은 서늘한 곳에(밀폐하여) 보관할 것.
(3) 오용(잘못사용)을 막고 품질의 보존을 위하여 다른 용기에 바꾸어 넣지 말 것.
---
문서3:스무스케어로션2.5%(히드로코르티손) :

### 주의사항(1)
6. 소아에 대한 투여

1) 12살 미만의 유ㆍ소아에게는 부신(콩팥위샘) 억제가 더 많이 일어날 수 있으므로 장기간 연속적인 국소 
요법은 가능하면 피한다.
2) 소아는 체중당 체표면적의 비율이 성인보다 크고, 피부장벽이 미성숙하여 성인에 비해 비례적으로 더 많
은 양이 흡수되어 HPA축 억제, 쿠싱증후군, 발육장애 등이 나타나기 쉬우므로 주의한다.
3) 기저귀 등은 밀봉붕대법과 같은 작용을 나타낼 수 있으므로 주의한다.
---
문서4:파로네프캡슐 :

### 주의사항(1)
1. 다음과 같은 사람은 이 약을 복용하지 말 것.
15세 미만의 소아
---
문서5:케로팝경고제(케토프로펜) :

### 주의사항(

In [ ]:
result_lst[1]

'케로팝경고제(케토프로펜)는 15세 미만의 소아에게 사용하지 말아야 합니다. 이는 약물의 주의사항에 명시되어 있습니다[[ref5]].'

In [ ]:
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 667/667 [20:47<00:00,  1.87s/it]


In [ ]:
def extract_ref_numbers(text):
    pattern = r'\[\[ref(\d+)\]\]'

    matches = re.findall(pattern, text)

    unique_numbers = sorted(set(map(int, matches)))

    return unique_numbers

In [ ]:
result_lst[0]

'케이포비돈아오다인액10%는 리튬치료를 받고 있는 환자에게 규칙적인 사용을 피해야 하며, 리튬치료와 병용할 경우 갑상선기능 항진효과가 나타날 수 있습니다. 따라서 리튬치료와 병용하는 것은 권장되지 않습니다[[ref1]].'

In [ ]:
result = extract_ref_numbers(result_lst[0])
print(result)

[1]


In [ ]:
extracted_ref_numbers = []
for sample in result_lst:
  extracted_ref_numbers.append(extract_ref_numbers(sample))

- 인용번호 없는 답변

In [ ]:
for idx, extracted_ref_number in enumerate(extracted_ref_numbers):
  if len(extracted_ref_number) == 0:
    print(idx)
    print(result_lst[idx])
    print('--')

34
검색된 문서에서 "파자임-95밀리그램이중정"의 정해진 용법에 대한 구체적인 내용은 확인할 수 없었습니다. 문서3에서는 이 약의 복용 시 주의할 사항으로 정해진 용법·용량을 잘 지킬 것을 언급하고 있지만, 구체적인 용법이나 용량에 대한 정보는 제공되지 않고 있습니다. 따라서 파자임-95밀리그램이중정의 정해진 용법에 대한 답변을 찾을 수 없습니다.
--
218
젠가드캡슐의 복용 방법에 대한 구체적인 정보는 검색된 문서에서 찾을 수 없었습니다. 따라서 젠가드캡슐의 복용 방법에 대한 답변을 제공할 수 없습니다.
--
381
엑스콜노즈정은 고혈압 환자가 복용해도 되는지에 대한 정보는 문서에서 명확히 언급되지 않았습니다. 그러나 문서1에서는 고혈압을 포함한 여러 질환이 있는 사람에게 주의가 필요하다고 명시하고 있습니다. 따라서 고혈압 환자가 엑스콜노즈정을 복용하기 전에 반드시 의사와 상담하는 것이 중요합니다. 추가적인 정보는 제공되지 않아 구체적인 복용 여부에 대한 답변을 찾을 수 없었습니다.
--
611
알두스캡슐(에르도스테인)은 다른 약물과의 상호작용에 대한 정보가 문서에서 제공되지 않았습니다. 따라서 알두스캡슐과의 상호작용에 대한 구체적인 내용을 찾을 수 없습니다.
--
661
아프소겐정 복용 시 주의할 사항에 대한 구체적인 내용은 문서에서 확인할 수 없습니다. 문서2에서는 아프소겐정의 복용 시 주의할 사항이 언급되지만, 구체적인 내용은 제공되지 않고 있습니다. 따라서 아프소겐정 복용 시 주의할 사항에 대한 답변을 찾을 수 없습니다.
--


In [ ]:
df_general['search_result'] = samples

In [ ]:
df_general['answer'] = result_lst

In [ ]:
df_general['extracted_ref_numbers'] = extracted_ref_numbers

In [ ]:
cited_df = df_general[~(df_general['extracted_ref_numbers'].apply(str) == '[]')].reset_index(drop=True)

In [ ]:
print('답변이 존재하는 데이터:', len(cited_df))

답변이 존재하는 데이터: 662


In [ ]:
cited_df[['question', 'search_result', 'answer', 'extracted_ref_numbers']]

,question,search_result,answer,extracted_ref_numbers
0,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?,[케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호...,케이포비돈아오다인액10%는 리튬치료를 받고 있는 환자에게 규칙적인 사용을 피해야 하...,[1]
1,케로팝경고제는 15세 미만 소아에게 사용 가능한가?,[크린팝카타플라스마 :\n\n### 주의사항(1)\n1. 다음 환자(부위)에는 투여...,케로팝경고제(케토프로펜)는 15세 미만의 소아에게 사용하지 말아야 합니다. 이는 약...,[5]
2,진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?,[징코하이정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n3. 다음...,"진코신정 80밀리그램은 출혈 장애가 있는 환자가 복용하기 전에 의사, 치과의사, 약...",[4]
3,파비스암브록솔염산염정은 어떻게 보관해야 하나요?,[파모트리정10밀리그램(파모티딘) :\n\n### 주의사항(1)\n5. 저장상의 주...,"파비스암브록솔염산염정은 다음과 같이 보관해야 합니다. 첫째, 어린이의 손이 닿지 않...",[5]
4,알피디연질캡슐은 수유 중 복용해도 되나요?,[이지페인릴리프연질캡슐 :\n\n### 주의사항(1)\n8. 임부 및 수유부에 대한...,알피디연질캡슐은 수유 중 복용을 피하는 것이 바람직합니다. 수유부의 투여에 대한 안...,[4]
...,...,...,...,...
657,마이노신캡슐은 갈락토오스 불내성 환자가 복용해도 되나요?,[브로신큐정 :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 이 약을 복용...,"마이노신캡슐은 유당(젖당) 수화물을 함유하고 있으므로, 갈락토오스 불내성 환자는 복...",[3]
658,벡크정은 만 8세 이상에게 어떻게 복용하나요?,[벡크정 :\n\n### 용법용량(1)\n용법용량\n만 8세 이상 및 성인 : 1회...,"벡크정은 만 8세 이상 및 성인에게 1회 1정, 1일 1회 복용하도록 되어 있습니다...",[1]
659,슬리프웰구강붕해필름은 임부가 복용해도 되나요?,[슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n...,슬리프웰구강붕해필름은 임부 및 수유부에게 사용이 권장되지 않습니다. 이 약을 복용하...,[1]
660,알파니푸록사지드캡슐의 부작용은 무엇인가요?,[알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르...,알파니푸록사지드캡슐의 부작용으로는 알레르기 반응이 나타날 수 있습니다[[ref1]].,[1]


- 업로드

In [ ]:
dataset = datasets.Dataset.from_pandas(cited_df)

In [ ]:
dataset.push_to_hub("otc-mrc-original-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/eddyfox8812/otc-mrc-original-question/commit/e6a4632e3d9848b742172be387c37868156c930d', commit_message='Upload dataset', commit_description='', oid='e6a4632e3d9848b742172be387c37868156c930d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eddyfox8812/otc-mrc-original-question', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eddyfox8812/otc-mrc-original-question'), pr_revision=None, pr_num=None)

## 3-3. no answer 질문 데이터 생성

In [ ]:
df_no = df_no.reset_index(drop=True)
df_no

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type
0,09a0b7c7451e,코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타(디클로페낙나트륨),warning,코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...,noanswer
1,62227c2b97f5,하이로손로션2.5%(히드로코르티손),하이로손로션2.5%(히드로코르티손),warning,하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...,noanswer
2,01a538aa8035,유카인크림(리도카인),유카인크림(리도카인),warning,유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,유카인크림은 어떻게 보관해야 하나요?,[엑스케인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주...,noanswer
3,e79531ef699d,씨이릴리프점안액(수출용),씨이릴리프점안액(수출용),dosage,"씨이릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1일 1-4회,...",씨이릴리프점안액은 하루 몇 회 점안하나요?,[어드밴스드릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1회 1...,noanswer
4,9aabec0ade2d,마이톡톡케어점안액,마이톡톡케어점안액,warning,마이톡톡케어점안액 :\n\n### 주의사항(1)\n사용상의주의사항\n1) 다음과 같...,마이톡톡케어점안액은 소프트콘택트렌즈 착용 시 사용 가능한가요?,[아이미루40이엑스점안액 :\n\n### 주의사항(1)\n3. 이 약을 사용하는 동...,noanswer
...,...,...,...,...,...,...,...,...
328,29502210f1b0,경남나프록센연질캡슐,경남나프록센연질캡슐,dosage,경남나프록센연질캡슐 :\n\n### 용법용량(1)\n용법용량\n1. 류마티양 관절염...,경남나프록센연질캡슐의 급성통풍 용법은?,[나프원큐연질캡슐(나프록센) :\n\n### 주의사항(1)\n2. 다음 환자에는 투...,noanswer
329,79bccd3248e9,폴리아린락토정,폴리아린락토정,warning,폴리아린락토정 :\n\n### 주의사항(1)\n3. 다음과 같은 사람(경우)은 이 ...,폴리아린락토정은 알레르기 병력이 있는 사람에게 어떻게 해야 하나요?,[락토란정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람(경우) 이 약의 ...,noanswer
330,c33a93c16f09,휘니볼(반하사심탕엑스과립),휘니볼(반하사심탕엑스과립),dosage,휘니볼(반하사심탕엑스과립) :\n\n### 용법용량(1)\n용법용량\n보통 성인 1...,휘니볼의 성인 복용량은 얼마인가요?,[폰트비정 :\n\n### 용법용량(1)\n용법용량\n만 12세 이상 및 성인 : ...,noanswer
331,c754c587d940,카프맥스정,카프맥스정,warning,카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,카프맥스정은 임부가 복용해도 되나요?,"[레피즈캡슐(니푸록사지드) :\n\n### 주의사항(1)\n6. 임부, 수유부, 가...",noanswer


In [ ]:
samples = []
for context, negative_samples in zip(df_no['context'].to_list(), df_no['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
user_prompts = []
for sample in samples:
  user_prompt = ''
  for idx, s in enumerate(sample):
    user_prompt = user_prompt + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [ ]:
system_prompt_question = """당신은 주어진 5개의 검색 문서로부터 답변이 불가능한 질문 5개를 생성해야 합니다.

### 구체적인 지시사항
1. 문서를 5개 드리겠습니다. 각 문서들과 연관된 질문을 하나씩 생성하십시오. 문서1, 문서2, 문서3, 문서4, 문서5 순서대로 합니다.
2. 질문은 각 문서와 연관된 질문이어야 합니다.
3. 답변의 순서는 연관 문서:, 질문:, 판단: 이 세 가지이며 5개의 문서가 주어지므로 총 5번 써야합니다.
4. 중요한 점은 생성된 각 질문은 5개의 그 어떠한 문서로도 답변이 불가능해야 한다는 것입니다.
5. 질문 작성 후에는 추가적인 코멘트는 작성하지 마십시오.
6. 답변의 형식은 다음과 같습니다. 연관 문서에서 질문을 생성하는데 참고한 문서를 적고 그 다음에 질문: 다음에 실제 질문을 생성하세요.
7. 판단: 과정에서 각 질문이 정말로 5개의 문서로 답변이 불가능한지 다시 한 번 판단하고 특정 문서로 답변이 가능한 것으로 판단되면 => False를 적고 답변이 불가능한게 맞다면 => True를 적으세요.
8. => False와 => True 다음에 추가적인 코멘트는 금지합니다.
9. 질문 내에서 약물을 언급할 때는 반드시 문서에 명시된 약의 이름을 사용해야합니다. 
 금지예시)
 이 약, 이 점안액, 약, 약물, 이 약물

 좋은 예시)
 타이레놀을 복용할 때 피해야 할 음식은 뭐야?
 리메부틴정의 효과는?
 가스모틴정의 유통기한은 언제까지인가요?
10. 질문은 짧게 작성하십시오. 예를 들어보겠습니다.
###
ex) 질문: 원종이 설립된 이후 해인사에서 어떤 특별한 행사나 의식이 처음으로 개최되었는지 알려주시겠습니까?
위와 같이 길게 작성하지 마십시오.
ex) 질문: 원종이 설립되고 해인사에서 처음 개최된 행사를 알려줘
위와 같이 짧게 작성하십시오.
###
10. 각 질문은 존댓말과 반말을 교차하면서 사용하십시오. 반말을 했다면 다음 질문은 가급적 존댓말을 하는 식입니다. 최대한 지켜주십시오.
11. 답변 형식은 다음과 같습니다.
ex)
답변:
연관 문서: 문서1
질문: (문서1과 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 문서 3에 연관된 내용 ~~가 있어서 답변이 가능합니다. => False

연관 문서: 문서2
질문: (문서2와 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

...중략...

연관 문서: 문서5
질문: (문서5와 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True
"""

- 5개로 테스트

In [ ]:
result_lst = []
for user_prompt in tqdm(user_prompts[:5]):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:23<00:00,  4.62s/it]


In [ ]:
print(result_lst[0])

답변:
연관 문서: 문서1  
질문: 코디톱쿨플라스타 사용 후 몇 시간 후에 증상이 나타날까요?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서2  
질문: 히드로코르티손이 소아에게 미치는 영향은 어떤 건지 알 수 있어?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서3  
질문: 모키쿨플라스타를 언제 처음 발견했는지 알아?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서4  
질문: 크린팝카타플라스마는 어떤 성분으로 만들어졌는지 알려줘.  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서5  
질문: 덴플러스포르테캡슐의 유통기한은 어떻게 되는지 알려주세요.  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  


In [ ]:
result_lst = []
for user_prompt in tqdm(user_prompts):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 333/333 [33:58<00:00,  6.12s/it]


In [ ]:
df_no['no_answer_question'] = result_lst

In [ ]:
false_samples = []
for sample in df_no['no_answer_question'].to_list():
  if '=> False' in sample:
    false_samples.append(sample)

print('False가 발생한 샘플의 수:', len(false_samples))

False가 발생한 샘플의 수: 45


In [ ]:
def remove_false_decisions(text):
    t = str(text).replace("\r\n", "\n").strip()

    blocks = re.split(r"\n(?=연관\s*문서\s*:)", t)
    blocks = [b.strip() for b in blocks if b.strip().startswith("연관")]

    keep = []
    for b in blocks:
        
        if re.search(r"=>\s*True\b", b):
            keep.append(b)

    return "\n\n".join(keep)

In [ ]:
def extract_questions(text):
    t = str(text).replace("\r\n", "\n")

    qs = re.findall(r"질문\s*:\s*(.*?)(?=\n판단\s*:)", t, flags=re.DOTALL)
    return [q.strip().replace("\n", " ") for q in qs]

In [ ]:
def select_random_question(questions_list):
    if not questions_list:
        return None
    return random.choice(questions_list)

In [ ]:
one_questions = []
for five_questions in df_no['no_answer_question'].to_list():
  true_questions = remove_false_decisions(five_questions)
  questions_list = extract_questions(true_questions)
  random_question = select_random_question(questions_list)
  one_questions.append(random_question)

In [ ]:
df_no[df_no['no_answer_question'].isna()]

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type,no_answer_question


In [ ]:
df_no['no_answer_question'] = one_questions

In [ ]:
one_questions = df_no['no_answer_question'].to_list()

## 3-4. no answer 데이터 생성

In [ ]:
system_prompt_answer =  """당신은 검색된 문서로부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 답변에서 참고자료의 내용을 인용한 경우, 답변 맨 끝에 인용한 문서 id를 추가하세요. 답변에 내용을 인용한 경우에만 작성해야 합니다. 인용한 게 없다면 쓰지마세요.
4. 문서 id 값은 검색 결과에서 가져온 것이며 두 개의 대괄호로 묶어야 하며 ref라는 키워드와 함께 작성해야 합니다. 예를 들어 특정 문단이 3번 문서에서 인용했다면 다음과 같이 작성하십시오.
예시) ~~~해서 했습니다. [[ref3]]
5. 주요 문장 또는 문단 뒤에 ref문서번호를 덧붙이는 것은 매우 중요합니다. 이는 반드시 지켜져야 합니다.
6. 문단 중간 중간에 인용이 들어가더라도 답변은 하나의 이어지는 평문으로 작성하십시오.
7. 주어진 문서는 검색 결과입니다. 따라서 질문과 연관없는 문서가 숨어있을 수 있으니 이를 인용하지 않도록 주의하십시오.
8. 검색 결과에 없는 내용은 답변으로 작성하지 마십시오. 가장 중요한 지시사항입니다. 따라서 인용 없이는 그 어떠한 설명도 제공할 수 없습니다.
9. 사용자는 한가지의 약물에 대해서만 질문합니다. 문서 내 약물을 정확히 찾아 답변하세요.
10. 반드시 사용자가 질문한 약물에 관한 문서만을 찾아 답변하세요. 이는 반드시 지켜져야 합니다.
11. 검색 결과는 모두 독립적인 문서들이며 반드시 연관되어져 있거나 순서대로 이어지는 문서들이 아닙니다.
12. 단, 질문에 대한 답이 검색 결과에 없다면 다음과 같이 답변을 작성하십시오. 반드시 아래의 형식을 따라야 합니다.
=> 답변 형식: 검색 결과에는 ~~의 반응을 찾을 수 없습니다.

예시를 드리겠습니다.

예시)
질문: 원종 설립 당시 일본 정부의 반응은 어떠했나요?
답변: 검색 결과에는 원종 설립 당시 일본 정부의 반응을 찾을 수 없습니다.

질문: 중국의 던전앤파이터 인기 게임 순위는 어떻게 되나요?
답변: 검색 결과에는 중국의 던전앤파이터 인기 게임 순위를 찾을 수 없습니다.

검색 결과에 질문에 대한 답변이 없다면 위 형식을 반드시 따르십시오.
"""

In [ ]:
samples = []
for context, negative_samples in zip(df_no['context'].to_list(), df_no['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
user_prompts = []
for sample in samples:
  user_prompt = '검색 결과:\n'
  for idx, s in enumerate(sample):
    user_prompt = user_prompt  + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [ ]:
user_prompt_for_answer = []
for multi_document_question, user_prompt in zip(one_questions, user_prompts):
  user_prompt_for_answer.append('질문:' + multi_document_question + '\n' + user_prompt + '답변:')

In [ ]:
print(user_prompt_for_answer[0])

질문:덴플러스포르테캡슐의 성분에 대해 설명해 줄래?
검색 결과:
문서1:코디톱쿨플라스타(디클로페낙나트륨) :

### 주의사항(1)
1. 다음과 같은 사람은 이 약을 사용하지 말 것.
1) 이 약 및 이 약의 성분에 과민반응의 병력이 있는 환자
2) 아스피린 또는 기타의 비스테로이드 소염제(항염증제)에 대한 천식(비스테로이드 소염제(항염증제)에 의
해 유발되는 천식 발작) 또는 그 병력이 있는 환자(중증(심한증상) 천식 발작을 유발할 우려가 있다.)
3) 아스피린 또는 기타의 비스테로이드 소염제(항염증제)에 과민반응 환자
4) 15세 미만의 어린이(어린이에 대해서는 안전성이 확립되어 있지 않다.(사용경험이 적다))
5) 임신 3기
---
문서2:코티손로션2.5%(히드로코르티손) :

### 주의사항(1)
6. 소아에 대한 투여

1) 12살 미만의 유ㆍ소아에게는 부신(콩팥위샘) 억제가 더 많이 일어날 수 있으므로 장기간 연속적인 국소 
요법은 가능하면 피한다.
2) 소아는 체중당 체표면적의 비율이 성인보다 크고, 피부장벽이 미성숙하여 성인에 비해 비례적으로 더 많
은 양이 흡수되어 HPA축 억제, 쿠싱증후군, 발육장애 등이 나타나기 쉬우므로 주의한다.
3) 기저귀 등은 밀봉붕대법과 같은 작용을 나타낼 수 있으므로 주의한다.
---
문서3:모키쿨플라스타 :

### 주의사항(1)
1. 다음과 같은 사람은 이 약을 사용하지 말 것
1) 이 약에 포함된 성분에 과민반응이 있는 사람
2) 30개월 이하의 소아
---
문서4:크린팝카타플라스마 :

### 주의사항(1)
1. 다음 환자(부위)에는 투여하지 말 것.
1) 30개월 이하의 유아
2) 습진, 옻 등에 의한 피부염, 상처부위
---
문서5:덴플러스포르테캡슐 :

### 주의사항(1)
1. 다음과 같은 사람은 이 약을 복용하지 말 것.
1) 15세 미만의 소아
2) 이 약은 유당을 함유하고 있으므로, 갈락토오스 불내성(galactose intolerance), Lapp 유당분해효소 결핍
증(Lapp lactase

In [ ]:
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 333/333 [06:54<00:00,  1.25s/it]


In [ ]:
df_no['no_answer'] = result_lst

- no answer 답변에 답변이 가능한경우

In [ ]:
df_no[df_no['no_answer'].str.contains('ref')]

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type,no_answer_question,no_answer
33,c531362fd9fd,세레지아정(세티리진염산염)(수출용),세레지아정(세티리진염산염)(수출용),warning,세레지아정(세티리진염산염)(수출용) :\n\n### 주의사항(1)\n3. 이 약을 ...,세레지아정은 과량의 알코올과 함께 투여해도 되나요?,[알지텍정(세티리진염산염) :\n\n### 주의사항(1)\n6. 기타 이 약의 복용...,noanswer,세레지아정을 복용하는 동안 어떤 특정 음료를 피해야 하나요?,세레지아정을 복용하는 동안 피해야 하는 특정 음료는 과량의 알코올입니다. 세레지아정...
127,e9ef5e3ddd46,브로신큐정,브로신큐정,warning,브로신큐정 :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 이 약을 복용하...,브로신큐정은 어떤 환자가 복용하면 안 되나요?,[브로멜라장용정(브로멜라인) :\n\n### 주의사항(1)\n1. 다음과 같은 사람...,noanswer,브로신큐정을 복용할 수 없는 대표적인 질병은 뭐야?,"브로신큐정은 유당을 함유하고 있으므로, 갈락토오스 불내성(galactose into..."
152,834098852afd,팜젠산화마그네슘정,팜젠산화마그네슘정,warning,팜젠산화마그네슘정 :\n\n### 주의사항(1)\n4. 다음과 같은 경우 이 약의 ...,팜젠산화마그네슘정 복용 후 어떤 증상이 나타나면 중지해야 하나요?,[파이네신캡슐(은교산) :\n\n### 주의사항(1)\n3. 다음과 같은 경우 이 ...,noanswer,경방평위산정의 일반적인 용법은 어떻게 되나?,"경방평위산정의 일반적인 용법에 따르면, 복용 중 다음과 같은 증상이 나타날 경우 즉..."
183,69603ec5b66a,아세드린정,아세드린정,warning,아세드린정 :\n\n### 주의사항(1)\n3. 이 약을 복용하는 동안 다음의 약을...,아세드린정 복용 중 어떤 약을 피해야 하나요?,[프리노즈에스정 :\n\n### 주의사항(1)\n(3) 이 약을 복용하는 동안 다음...,noanswer,에카린에이정 사용 시 주의해야 할 특정 부작용은 무엇인가요?,"에카린에이정 사용 시 주의해야 할 특정 부작용으로는 진해거담제, 다른 감기약, 해열..."
234,048593286bf9,세토펜정(아세트아미노펜),세토펜정(아세트아미노펜),warning,세토펜정(아세트아미노펜) :\n\n### 주의사항(1)\n4. 다음과 같을 경우 이...,세토펜정 복용 시 혈액 관련 주의사항은?,[세토펜정80밀리그램(아세트아미노펜) :\n\n### 주의사항(1)\n2. 다음과 ...,noanswer,세토펜정이 간 손상 외에 어떤 문제를 일으킬 수 있는지 알려줄 수 있어?,세토펜정은 간 손상 외에도 여러 가지 문제를 일으킬 수 있습니다. 이 약물의 복용으...


In [ ]:
df_no = df_no[~df_no['no_answer'].str.contains('ref')].reset_index(drop=True)

In [ ]:
sample_check = []
for sample in df_no['no_answer'].to_list():
  if '검색 결과' not in sample:
    sample_check.append(sample)

In [ ]:
len(sample_check)

0

In [ ]:
extracted_ref_numbers = []
for sample in df_no['no_answer'].to_list():
  extracted_ref_numbers.append(extract_ref_numbers(sample))

In [ ]:
df_no['extracted_ref_numbers'] = extracted_ref_numbers

In [ ]:
df_no[~(df_no['extracted_ref_numbers'].apply(str) == '[]')].reset_index(drop=True)

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type,no_answer_question,no_answer,extracted_ref_numbers


In [ ]:
samples = []
for context, negative_samples in zip(df_no['context'].to_list(), df_no['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
df_no['search_result'] = samples

In [ ]:
df_no

,drug_id,drug_name_key,drug_name,qa_type_hint,context,question,negative_samples,label_type,no_answer_question,no_answer,extracted_ref_numbers,search_result
0,09a0b7c7451e,코디톱쿨플라스타(디클로페낙나트륨),코디톱쿨플라스타(디클로페낙나트륨),warning,코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...,코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...,noanswer,덴플러스포르테캡슐의 성분에 대해 설명해 줄래?,검색 결과에는 덴플러스포르테캡슐의 성분에 대한 설명을 찾을 수 없습니다.,[],[코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같...
1,62227c2b97f5,하이로손로션2.5%(히드로코르티손),하이로손로션2.5%(히드로코르티손),warning,하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...,하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?,[코티손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n6. 소아에 대...,noanswer,케어스킨로션을 사용하면 생리통이 완화될까요?,검색 결과에는 케어스킨로션을 사용하면 생리통이 완화될 수 있는지에 대한 정보를 찾을...,[],[하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환...
2,01a538aa8035,유카인크림(리도카인),유카인크림(리도카인),warning,유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...,유카인크림은 어떻게 보관해야 하나요?,[엑스케인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주...,noanswer,테라미플러스크림을 어느 약국에서 구할 수 있는지 아는 사람 있어?,검색 결과에는 테라미플러스크림을 어느 약국에서 구할 수 있는지 찾을 수 없습니다.,[],[유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의...
3,e79531ef699d,씨이릴리프점안액(수출용),씨이릴리프점안액(수출용),dosage,"씨이릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1일 1-4회,...",씨이릴리프점안액은 하루 몇 회 점안하나요?,[어드밴스드릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1회 1...,noanswer,클리어드롭점안액의 유통기한은 얼마인가요?,검색 결과에는 클리어드롭점안액의 유통기한을 찾을 수 없습니다.,[],[씨이릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1일 1-4회...
4,9aabec0ade2d,마이톡톡케어점안액,마이톡톡케어점안액,warning,마이톡톡케어점안액 :\n\n### 주의사항(1)\n사용상의주의사항\n1) 다음과 같...,마이톡톡케어점안액은 소프트콘택트렌즈 착용 시 사용 가능한가요?,[아이미루40이엑스점안액 :\n\n### 주의사항(1)\n3. 이 약을 사용하는 동...,noanswer,모이스뷰점안액의 사용 금지는 어떤 제품으로부터 영향을 받나요?,검색 결과에는 모이스뷰점안액의 사용 금지와 관련하여 어떤 제품으로부터 영향을 받는지...,[],[마이톡톡케어점안액 :\n\n### 주의사항(1)\n사용상의주의사항\n1) 다음과 ...
...,...,...,...,...,...,...,...,...,...,...,...,...
323,29502210f1b0,경남나프록센연질캡슐,경남나프록센연질캡슐,dosage,경남나프록센연질캡슐 :\n\n### 용법용량(1)\n용법용량\n1. 류마티양 관절염...,경남나프록센연질캡슐의 급성통풍 용법은?,[나프원큐연질캡슐(나프록센) :\n\n### 주의사항(1)\n2. 다음 환자에는 투...,noanswer,콜록프로코프의 유통기한은 얼마나 돼?,검색 결과에는 콜록프로코프의 유통기한을 찾을 수 없습니다.,[],[경남나프록센연질캡슐 :\n\n### 용법용량(1)\n용법용량\n1. 류마티양 관절...
324,79bccd3248e9,폴리아린락토정,폴리아린락토정,warning,폴리아린락토정 :\n\n### 주의사항(1)\n3. 다음과 같은 사람(경우)은 이 ...,폴리아린락토정은 알레르기 병력이 있는 사람에게 어떻게 해야 하나요?,[락토란정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람(경우) 이 약의 ...,noanswer,락토란정을 복용하던 중 입이 마르지 않으면 괜찮은 거지?,검색 결과에는 락토란정을 복용하던 중 입이 마르지 않으면 괜찮은지에 대한 내용을 찾...,[],[폴리아린락토정 :\n\n### 주의사항(1)\n3. 다음과 같은 사람(경우)은 이...
325,c33a93c16f09,휘니볼(반하사심탕엑스과립),휘니볼(반하사심탕엑스과립),dosage,휘니볼(반하사심탕엑스과립) :\n\n### 용법용량(1)\n용법용량\n보통 성인 1...,휘니볼의 성인 복용량은 얼마인가요?,[폰트비정 :\n\n### 용법용량(1)\n용법용량\n만 12세 이상 및 성인 : ...,noanswer,비장원과립의 유통기한은 어떻게 되지?,검색 결과에는 비장원과립의 유통기한을 찾을 수 없습니다.,[],[휘니볼(반하사심탕엑스과립) :\n\n### 용법용량(1)\n용법용량\n보통 성인 ...
326,c754c587d940,카프맥스정,카프맥스정,warning,카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용하...,카프맥스정은 임부가 복용해도 되나요?,"[레피즈캡슐(니푸록사지드) :\n\n### 주의사항(1)\n6. 임부, 수유부, 가...",noanswer,한풍보중익기탕산을 사용할 때 필요한 사전 절차는 뭐야?,검색 결과에는 한풍보중익기탕산을 사용할 때 필요한 사전 절차에 대한 내용을 찾을 수...,[],[카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용...


# 4. 최종 데이터셋

In [ ]:
dataset1 = load_dataset("eddyfox8812/otc-mrc-original-question")
df1 = dataset1['train'].to_pandas()

print(df1.head())

README.md:   0%|          | 0.00/736 [00:00<?, ?B/s]

C:\Users\eddy_\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eddy_\.cache\huggingface\hub\datasets--eddyfox8812--otc-mrc-original-question. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/935k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/662 [00:00<?, ? examples/s]

        drug_id          drug_name_key              drug_name qa_type_hint  \
0  e227ace54e1f  케이포비돈아오다인액10%(포비돈요오드)  케이포비돈아오다인액10%(포비돈요오드)      warning   
1  7304a9a6f07c          케로팝경고제(케토프로펜)          케로팝경고제(케토프로펜)      warning   
2  f4c544a98592    진코신정80밀리그램(은행엽건조엑스)    진코신정80밀리그램(은행엽건조엑스)      warning   
3  1eae41b300f9       파비스암브록솔염산염정(수출용)       파비스암브록솔염산염정(수출용)      warning   
4  3b49b5e28d21  알피디연질캡슐1마이크로그램(알파칼시돌)  알피디연질캡슐1마이크로그램(알파칼시돌)      warning   

                                             context  \
0  케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호작...   
1  케로팝경고제(케토프로펜) :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 ...   
2  진코신정80밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n2. 다음과 같...   
3  파비스암브록솔염산염정(수출용) :\n\n### 주의사항(1)\n6. 저장상의 주의사...   
4  알피디연질캡슐1마이크로그램(알파칼시돌) :\n\n### 주의사항(1)\n3. 이 약...   

                              question  \
0      케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?   
1         케로팝경고제는 15세 미만 소아에게 사용 가능한가?   
2  진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?   
3           파비스암브록솔염산염정은 어

In [ ]:
dataset2 = load_dataset("eddyfox8812/otc-mrc-no-answer-question")
df2 = dataset2['train'].to_pandas()

print(df2.head())

README.md:   0%|          | 0.00/793 [00:00<?, ?B/s]

C:\Users\eddy_\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eddy_\.cache\huggingface\hub\datasets--eddyfox8812--otc-mrc-no-answer-question. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/662k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/328 [00:00<?, ? examples/s]

        drug_id        drug_name_key            drug_name qa_type_hint  \
0  09a0b7c7451e   코디톱쿨플라스타(디클로페낙나트륨)   코디톱쿨플라스타(디클로페낙나트륨)      warning   
1  62227c2b97f5  하이로손로션2.5%(히드로코르티손)  하이로손로션2.5%(히드로코르티손)      warning   
2  01a538aa8035          유카인크림(리도카인)          유카인크림(리도카인)      warning   
3  e79531ef699d        씨이릴리프점안액(수출용)        씨이릴리프점안액(수출용)       dosage   
4  9aabec0ade2d            마이톡톡케어점안액            마이톡톡케어점안액      warning   

                                             context  \
0  코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같은...   
1  하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환자...   
2  유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의\...   
3  씨이릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1일 1-4회,...   
4  마이톡톡케어점안액 :\n\n### 주의사항(1)\n사용상의주의사항\n1) 다음과 같...   

                             question  \
0     코디톱쿨플라스타는 15세 미만 어린이가 사용해도 되나요?   
1    하이로손로션2.5%는 고막 천공 환자에게 사용 가능한가요?   
2                유카인크림은 어떻게 보관해야 하나요?   
3             씨이릴리프점안액은 하루 몇 회 점안하나요?   
4  마이톡톡케어점안액은

In [ ]:
df1co = df1[['question', 'search_result', 'answer', 'extracted_ref_numbers']]
df1co

,question,search_result,answer,extracted_ref_numbers
0,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?,[케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호...,케이포비돈아오다인액10%는 리튬치료를 받고 있는 환자에게 규칙적인 사용을 피해야 하...,[1]
1,케로팝경고제는 15세 미만 소아에게 사용 가능한가?,[크린팝카타플라스마 :\n\n### 주의사항(1)\n1. 다음 환자(부위)에는 투여...,케로팝경고제(케토프로펜)는 15세 미만의 소아에게 사용하지 말아야 합니다. 이는 약...,[5]
2,진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?,[징코하이정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n3. 다음...,"진코신정 80밀리그램은 출혈 장애가 있는 환자가 복용하기 전에 의사, 치과의사, 약...",[4]
3,파비스암브록솔염산염정은 어떻게 보관해야 하나요?,[파모트리정10밀리그램(파모티딘) :\n\n### 주의사항(1)\n5. 저장상의 주...,"파비스암브록솔염산염정은 다음과 같이 보관해야 합니다. 첫째, 어린이의 손이 닿지 않...",[5]
4,알피디연질캡슐은 수유 중 복용해도 되나요?,[이지페인릴리프연질캡슐 :\n\n### 주의사항(1)\n8. 임부 및 수유부에 대한...,알피디연질캡슐은 수유 중 복용을 피하는 것이 바람직합니다. 수유부의 투여에 대한 안...,[4]
...,...,...,...,...
657,마이노신캡슐은 갈락토오스 불내성 환자가 복용해도 되나요?,[브로신큐정 :\n\n### 주의사항(1)\n1. 다음과 같은 사람은 이 약을 복용...,"마이노신캡슐은 유당(젖당) 수화물을 함유하고 있으므로, 갈락토오스 불내성 환자는 복...",[3]
658,벡크정은 만 8세 이상에게 어떻게 복용하나요?,[벡크정 :\n\n### 용법용량(1)\n용법용량\n만 8세 이상 및 성인 : 1회...,"벡크정은 만 8세 이상 및 성인에게 1회 1정, 1일 1회 복용하도록 되어 있습니다...",[1]
659,슬리프웰구강붕해필름은 임부가 복용해도 되나요?,[슬리프웰구강붕해필름3밀리그램(멜라토닌)(수출용) :\n\n### 주의사항(1)\n...,슬리프웰구강붕해필름은 임부 및 수유부에게 사용이 권장되지 않습니다. 이 약을 복용하...,[1]
660,알파니푸록사지드캡슐의 부작용은 무엇인가요?,[알파니푸록사지드캡슐(수출용) :\n\n### 주의사항(1)\n2. 부작용\n알레르...,알파니푸록사지드캡슐의 부작용으로는 알레르기 반응이 나타날 수 있습니다[[ref1]].,[1]


In [ ]:
df2co = df2[['no_answer_question', 'search_result', 'no_answer', 'extracted_ref_numbers']]
df2co

,no_answer_question,search_result,no_answer,extracted_ref_numbers
0,덴플러스포르테캡슐의 성분에 대해 설명해 줄래?,[코디톱쿨플라스타(디클로페낙나트륨) :\n\n### 주의사항(1)\n1. 다음과 같...,검색 결과에는 덴플러스포르테캡슐의 성분에 대한 설명을 찾을 수 없습니다.,[]
1,케어스킨로션을 사용하면 생리통이 완화될까요?,[하이로손로션2.5%(히드로코르티손) :\n\n### 주의사항(1)\n1. 다음 환...,검색 결과에는 케어스킨로션을 사용하면 생리통이 완화될 수 있는지에 대한 정보를 찾을...,[]
2,테라미플러스크림을 어느 약국에서 구할 수 있는지 아는 사람 있어?,[유카인크림(리도카인) :\n\n### 주의사항(1)\n5. 저장 및 취급상의 주의...,검색 결과에는 테라미플러스크림을 어느 약국에서 구할 수 있는지 찾을 수 없습니다.,[]
3,클리어드롭점안액의 유통기한은 얼마인가요?,[씨이릴리프점안액(수출용) :\n\n### 용법용량(1)\n용법용량\n1일 1-4회...,검색 결과에는 클리어드롭점안액의 유통기한을 찾을 수 없습니다.,[]
4,모이스뷰점안액의 사용 금지는 어떤 제품으로부터 영향을 받나요?,[마이톡톡케어점안액 :\n\n### 주의사항(1)\n사용상의주의사항\n1) 다음과 ...,검색 결과에는 모이스뷰점안액의 사용 금지와 관련하여 어떤 제품으로부터 영향을 받는지...,[]
...,...,...,...,...
323,콜록프로코프의 유통기한은 얼마나 돼?,[경남나프록센연질캡슐 :\n\n### 용법용량(1)\n용법용량\n1. 류마티양 관절...,검색 결과에는 콜록프로코프의 유통기한을 찾을 수 없습니다.,[]
324,락토란정을 복용하던 중 입이 마르지 않으면 괜찮은 거지?,[폴리아린락토정 :\n\n### 주의사항(1)\n3. 다음과 같은 사람(경우)은 이...,검색 결과에는 락토란정을 복용하던 중 입이 마르지 않으면 괜찮은지에 대한 내용을 찾...,[]
325,비장원과립의 유통기한은 어떻게 되지?,[휘니볼(반하사심탕엑스과립) :\n\n### 용법용량(1)\n용법용량\n보통 성인 ...,검색 결과에는 비장원과립의 유통기한을 찾을 수 없습니다.,[]
326,한풍보중익기탕산을 사용할 때 필요한 사전 절차는 뭐야?,[카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용...,검색 결과에는 한풍보중익기탕산을 사용할 때 필요한 사전 절차에 대한 내용을 찾을 수...,[]


In [ ]:
df1co['type'] = 'mrc_question'
df2co['type'] = 'no_answer'

C:\Users\eddy_\AppData\Local\Temp\ipykernel_11032\2223422694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1co['type'] = 'mrc_question'
C:\Users\eddy_\AppData\Local\Temp\ipykernel_11032\2223422694.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2co['type'] = 'no_answer'


In [ ]:
df1co.columns = ["question", "search_result", "answer", "extracted_ref_numbers", "type"]
df2co.columns = ["question", "search_result", "answer", "extracted_ref_numbers", "type"]

In [ ]:
df_final = pd.concat([df1co, df2co], ignore_index=True)
df_final

,question,search_result,answer,extracted_ref_numbers,type
0,케이포비돈아오다인액10%는 리튬치료와 병용할 수 있나요?,[케이포비돈아오다인액10%(포비돈요오드) :\n\n### 주의사항(1)\n5. 상호...,케이포비돈아오다인액10%는 리튬치료를 받고 있는 환자에게 규칙적인 사용을 피해야 하...,[1],mrc_question
1,케로팝경고제는 15세 미만 소아에게 사용 가능한가?,[크린팝카타플라스마 :\n\n### 주의사항(1)\n1. 다음 환자(부위)에는 투여...,케로팝경고제(케토프로펜)는 15세 미만의 소아에게 사용하지 말아야 합니다. 이는 약...,[5],mrc_question
2,진코신정80밀리그램은 출혈 장애가 있는 환자가 복용해도 되나요?,[징코하이정240밀리그램(은행엽건조엑스) :\n\n### 주의사항(1)\n3. 다음...,"진코신정 80밀리그램은 출혈 장애가 있는 환자가 복용하기 전에 의사, 치과의사, 약...",[4],mrc_question
3,파비스암브록솔염산염정은 어떻게 보관해야 하나요?,[파모트리정10밀리그램(파모티딘) :\n\n### 주의사항(1)\n5. 저장상의 주...,"파비스암브록솔염산염정은 다음과 같이 보관해야 합니다. 첫째, 어린이의 손이 닿지 않...",[5],mrc_question
4,알피디연질캡슐은 수유 중 복용해도 되나요?,[이지페인릴리프연질캡슐 :\n\n### 주의사항(1)\n8. 임부 및 수유부에 대한...,알피디연질캡슐은 수유 중 복용을 피하는 것이 바람직합니다. 수유부의 투여에 대한 안...,[4],mrc_question
...,...,...,...,...,...
985,콜록프로코프의 유통기한은 얼마나 돼?,[경남나프록센연질캡슐 :\n\n### 용법용량(1)\n용법용량\n1. 류마티양 관절...,검색 결과에는 콜록프로코프의 유통기한을 찾을 수 없습니다.,[],no_answer
986,락토란정을 복용하던 중 입이 마르지 않으면 괜찮은 거지?,[폴리아린락토정 :\n\n### 주의사항(1)\n3. 다음과 같은 사람(경우)은 이...,검색 결과에는 락토란정을 복용하던 중 입이 마르지 않으면 괜찮은지에 대한 내용을 찾...,[],no_answer
987,비장원과립의 유통기한은 어떻게 되지?,[휘니볼(반하사심탕엑스과립) :\n\n### 용법용량(1)\n용법용량\n보통 성인 ...,검색 결과에는 비장원과립의 유통기한을 찾을 수 없습니다.,[],no_answer
988,한풍보중익기탕산을 사용할 때 필요한 사전 절차는 뭐야?,[카프맥스정 :\n\n### 주의사항(1)\n4. 다음과 같은 사람은 이 약을 복용...,검색 결과에는 한풍보중익기탕산을 사용할 때 필요한 사전 절차에 대한 내용을 찾을 수...,[],no_answer


- 업로드

In [ ]:
dataset = datasets.Dataset.from_pandas(df_final)
dataset.push_to_hub("otc-mrc-ko-rag-dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/eddyfox8812/otc-mrc-ko-rag-dataset/commit/0025032d8c3656b6c84d1dcbac27b3c1cd9ca861', commit_message='Upload dataset', commit_description='', oid='0025032d8c3656b6c84d1dcbac27b3c1cd9ca861', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/eddyfox8812/otc-mrc-ko-rag-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='eddyfox8812/otc-mrc-ko-rag-dataset'), pr_revision=None, pr_num=None)